# 4.0.8 Sampling around Event Boundaries Fixed

A notebook that aims to credo 4.0.4 but with the correct data assembled in 4.0.6, and do so for both 64 Dim and I3D.

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,h5py,pandas,matplotlib,seaborn,jupyterlab,lab

Wed Feb 26 2020 16:05:56 

CPython 3.6.10
IPython 7.12.0

torch 1.2.0
torchvision 0.1.8
cv2 3.4.2
h5py 2.8.0
pandas 1.0.1
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.29.g1bb7899.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-76-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : 1bb789928298409106529d6e8942da764a7e8865
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode
%matplotlib inline

## Imports

In [4]:
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [5]:
# Constants to be used throughout the package
%aimport lab.index
from lab.index import DIR_DATA_INT, DIR_DATA_RAW
%aimport lab.breakfast.constants
from lab.breakfast.constants import SEED
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import (DIR_BREAKFAST, 
                                 DIR_RAW_BREAKFAST,
                                 DIR_BREAKFAST_DATA, 
                                 DIR_COARSE_SEG, 
                                 DIR_FINE_SEG,
                                )

## Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [6]:
sns.set()

In [7]:
master_df = pd.read_csv(str(DIR_RAW_BREAKFAST / 'master_meta.csv'))
master_df

,path_videos,length,patient,action,camera,channel,id,path_coarse,path_fine,path_64dim_fv,path_i3d
0,/media/data_cifs2/apra/work/labwork/data/exter...,2669,39,salat,cam01,NaN,39_salat_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1,/media/data_cifs2/apra/work/labwork/data/exter...,1304,39,tea,cam01,NaN,39_tea_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
2,/media/data_cifs2/apra/work/labwork/data/exter...,8990,39,pancake,cam01,NaN,39_pancake_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
3,/media/data_cifs2/apra/work/labwork/data/exter...,699,39,cereals,cam01,NaN,39_cereals_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
4,/media/data_cifs2/apra/work/labwork/data/exter...,2890,39,friedegg,cam01,NaN,39_friedegg_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
...,...,...,...,...,...,...,...,...,...,...,...
1984,/media/data_cifs2/apra/work/labwork/data/exter...,1123,5,coffee,cam01,NaN,5_coffee_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1985,/media/data_cifs2/apra/work/labwork/data/exter...,2795,5,juice,stereo,0.0,5_juice_stereo_0,NaN,NaN,NaN,NaN
1986,/media/data_cifs2/apra/work/labwork/data/exter...,1084,5,milk,stereo,0.0,5_milk_stereo_0,NaN,NaN,NaN,NaN
1987,/media/data_cifs2/apra/work/labwork/data/exter...,2795,5,juice,stereo,1.0,5_juice_stereo_1,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...


## Getting Start Frames and Number of Segments

In [8]:
with open(str(master_df.path_coarse.iloc[0]), 'r') as f:
    for l in f:
        print(l)

1-31 SIL  

32-1366 cut_fruit  

1367-1421 take_bowl  

1422-1551 put_fruit2bowl  

1552-1671 cut_fruit  

1672-2071 peel_fruit  

2072-2401 cut_fruit  

2402-2511 put_fruit2bowl  

2512-2669 SIL  



In [9]:
with open(str(master_df.path_coarse.iloc[0]), 'r') as f:
    starts = [line.split('-')[0] for line in f][1:]
    print(starts)

['32', '1367', '1422', '1552', '1672', '2072', '2402', '2512']


In [10]:
coarse_sub_df = master_df[master_df.path_coarse.notnull()]
coarse_sub_df

,path_videos,length,patient,action,camera,channel,id,path_coarse,path_fine,path_64dim_fv,path_i3d
0,/media/data_cifs2/apra/work/labwork/data/exter...,2669,39,salat,cam01,NaN,39_salat_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1,/media/data_cifs2/apra/work/labwork/data/exter...,1304,39,tea,cam01,NaN,39_tea_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
2,/media/data_cifs2/apra/work/labwork/data/exter...,8990,39,pancake,cam01,NaN,39_pancake_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
3,/media/data_cifs2/apra/work/labwork/data/exter...,699,39,cereals,cam01,NaN,39_cereals_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
4,/media/data_cifs2/apra/work/labwork/data/exter...,2890,39,friedegg,cam01,NaN,39_friedegg_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
...,...,...,...,...,...,...,...,...,...,...,...
1982,/media/data_cifs2/apra/work/labwork/data/exter...,1186,5,cereals,cam01,NaN,5_cereals_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1983,/media/data_cifs2/apra/work/labwork/data/exter...,2795,5,juice,cam01,NaN,5_juice_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1984,/media/data_cifs2/apra/work/labwork/data/exter...,1123,5,coffee,cam01,NaN,5_coffee_cam01,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...
1987,/media/data_cifs2/apra/work/labwork/data/exter...,2795,5,juice,stereo,1.0,5_juice_stereo_1,/media/data_cifs2/apra/work/labwork/data/exter...,NaN,/media/data_cifs2/apra/work/labwork/data/exter...,/media/data_cifs2/apra/work/labwork/data/exter...


In [11]:
ids, n_coarse_segments, coarse_segment_starts = [], [], []

for id, path in coarse_sub_df[['id', 'path_coarse']].values:
    with open(str(path), 'r') as file:
        ids.append(id)
        coarse_segment_starts.append([int(line.split('-')[0]) for line in file][1:])
        n_coarse_segments.append(len(coarse_segment_starts[-1]))

In [12]:
len(ids)

1677

In [13]:
ids[:5]

['39_salat_cam01',
 '39_tea_cam01',
 '39_pancake_cam01',
 '39_cereals_cam01',
 '39_friedegg_cam01']

## Convenience DFs

In [14]:
coarse_df = pd.DataFrame({'id' : ids})
coarse_df['n_coarse_segments'] = n_coarse_segments
coarse_df['coarse_segment_starts'] = coarse_segment_starts
coarse_df = coarse_df.merge(master_df[['id', 'length']], how='left', on='id')

In [15]:
coarse_64d_fv_df = coarse_df.merge(master_df[['id', 'path_64dim_fv']].dropna(), 
                                   how='inner', on='id')

In [16]:
coarse_i3d_df = coarse_df.merge(master_df[['id', 'path_i3d']].dropna(), 
                                how='inner', on='id')

## Making the List of Event Indices

In [18]:
# Select this number of frames
n_frames = 64
buffer = 15 # Assume people correctly label within a second of event

Should ignore indices that start with 2

### 64 FV Events

In [20]:
np.random.seed(SEED)

all_clip_ranges = []
all_clip_diffs = []
start_clip_idcs = []

for n, starts, length in coarse_64d_fv_df[
    ['n_coarse_segments', 'coarse_segment_starts', 'length']].values:
    # Ignore last segment which is start of SIL
    idcs_to_use = starts[:-1]
    # 2 Shows up disproportionally more than others. Ignore it
    if 2 in idcs_to_use:
        idcs_to_use = idcs_to_use[1:]
    
    # Move on if there are no events left to use
    if not idcs_to_use:
        all_clip_ranges.append([])
        all_clip_diffs.append(np.array([]))
        start_clip_idcs.append([])
        continue 
    
    clip_starts, clip_ranges = [], []
    
    # Calculate differences between each of the start points
    diffs = np.diff(idcs_to_use)
    for i, diff in enumerate(diffs):
        # If the value is greater than n_frames + 2*buffer,
        if diff > n_frames + 2*buffer:
            # Decide how many clips to create based on how many times larger
            n_clips = (diff - 2*buffer) // n_frames
            
            # use that starting point + buffer
            diff_clip_starts = [idcs_to_use[i] + 2*buffer + clip*n_frames
                                for clip in range(n_clips)]
            # Build the ranges
            clip_ranges += [np.arange(start, start+n_frames)
                            for start in diff_clip_starts]
            clip_starts += diff_clip_starts
    
    # List of the full 64 frame clip indices
    all_clip_ranges.append(clip_ranges)
    # And the starting idx of the clip
    start_clip_idcs.append(clip_starts)
    # All the diffs
    all_clip_diffs.append(diffs)
    
print(n_frames, buffer)
# print(idcs_to_use)
# print(diffs)
# print(n_clips)
# print(len(clip_starts))
# print(len(clip_ranges))
print(len(all_clip_ranges))
print(len(start_clip_idcs))

64 15
1474
1474


#### Saving the Interim Data

In [24]:
path_interim_data = DIR_DATA_INT / 'breakfast/video_clips/nonevent_clips/'
if not path_interim_data.exists():
    path_interim_data.mkdir(parents=True)
    
path_interim_64fv = path_interim_data / '64dim_fv_clips'
if not path_interim_64fv.exists():
    path_interim_64fv.mkdir(parents=True)

In [27]:
with open(str(path_interim_64fv / f'nonevent_clip_diffs_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(all_clip_diffs, filehandle)

In [28]:
with open(str(path_interim_64fv / f'nonevent_clip_ranges_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(all_clip_ranges, filehandle)

In [29]:
with open(str(path_interim_64fv / f'nonevent_clip_starts_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(start_clip_idcs, filehandle)

#### Creating the 64 Dim FV Clips

In [33]:
coarse_64d_fv_df['all_clip_ranges'] = all_clip_ranges
coarse_64d_fv_df['start_clip_idcs'] = start_clip_idcs
coarse_64d_fv_df['all_clip_diffs'] = all_clip_diffs

In [35]:
path_clips = path_interim_64fv / 'clips'
if not path_clips.exists():
    path_clips.mkdir(parents=True)
path_clips

PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips')

In [37]:
for id, path, clip_ranges, starts in coarse_64d_fv_df[[
    'id',
    'path_64dim_fv',
    'all_clip_ranges', 
    'start_clip_idcs',
]].values:
    file_array = pd.read_table(str(path), index_col=0, header=0).to_numpy()
    for clip_range, start in zip(clip_ranges, starts):
        path_out = path_clips / f'{id}_start_{start}_seed_{SEED}.npy'
        np.save(str(path_out), file_array[clip_range, :])

### I3D Events

In [39]:
np.random.seed(SEED)

all_clip_ranges = []
all_clip_diffs = []
start_clip_idcs = []

for n, starts, length in tqdm(coarse_i3d_df[[
    'n_coarse_segments', 
    'coarse_segment_starts', 
    'length'
]].values):
    # Ignore last segment which is start of SIL
    idcs_to_use = starts[:-1]
    # 2 Shows up disproportionally more than others. Ignore it
    if 2 in idcs_to_use:
        idcs_to_use = idcs_to_use[1:]
    
    # Move on if there are no events left to use
    if not idcs_to_use:
        all_clip_ranges.append([])
        all_clip_diffs.append(np.array([]))
        start_clip_idcs.append([])
        continue 
    
    clip_starts, clip_ranges = [], []
    
    # Calculate differences between each of the start points
    diffs = np.diff(idcs_to_use)
    for i, diff in enumerate(diffs):
        # If the value is greater than n_frames + 2*buffer,
        if diff > n_frames + 2*buffer:
            # Decide how many clips to create based on how many times larger
            n_clips = (diff - 2*buffer) // n_frames
            
            # use that starting point + buffer
            diff_clip_starts = [idcs_to_use[i] + 2*buffer + clip*n_frames
                                for clip in range(n_clips)]
            # Build the ranges
            clip_ranges += [np.arange(start, start+n_frames)
                            for start in diff_clip_starts]
            clip_starts += diff_clip_starts
    
    # List of the full 64 frame clip indices
    all_clip_ranges.append(clip_ranges)
    # And the starting idx of the clip
    start_clip_idcs.append(clip_starts)
    # All the diffs
    all_clip_diffs.append(diffs)

print(n_frames, buffer)
# print(idcs_to_use)
# print(diffs)
# print(n_clips)
# print(len(clip_starts))
# print(len(clip_ranges))
print(len(all_clip_ranges))
print(len(start_clip_idcs))

100%|██████████| 1576/1576 [00:00<00:00, 12069.30it/s]

64 15
1576
1576


### Saving the Interim Data

In [40]:
path_interim_data = DIR_DATA_INT / 'breakfast/video_clips/nonevent_clips/'
if not path_interim_data.exists():
    path_interim_data.mkdir(parents=True)
    
path_interim_i3d = path_interim_data / 'i3d_clips'
if not path_interim_i3d.exists():
    path_interim_i3d.mkdir(parents=True)

In [41]:
with open(str(path_interim_i3d / f'nonevent_clip_diffs_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(all_clip_diffs, filehandle)

In [42]:
with open(str(path_interim_i3d / f'nonevent_clip_ranges_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(all_clip_ranges, filehandle)

In [43]:
with open(str(path_interim_i3d / f'nonevent_clip_starts_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(start_clip_idcs, filehandle)

### Creating the I3D Clips

In [45]:
coarse_i3d_df['all_clip_ranges'] = all_clip_ranges
coarse_i3d_df['start_clip_idcs'] = start_clip_idcs
coarse_i3d_df['all_clip_diffs'] = all_clip_diffs

In [46]:
path_clips = path_interim_i3d / 'clips'
if not path_clips.exists():
    path_clips.mkdir(parents=True)
path_clips

PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips')

In [48]:
for id, path, clip_ranges, starts in tqdm(coarse_i3d_df[[
    'id',
    'path_i3d',
    'all_clip_ranges', 
    'start_clip_idcs', 
]].values):
    file_array = np.load(str(path))
    for clip_range, start in zip(clip_ranges, starts):
        path_out = path_clips / f'{id}_start_{start}_seed_{SEED}.npy'
        np.save(str(path_out), file_array[clip_range, :])

100%|██████████| 1576/1576 [33:07<00:00,  1.26s/it] 


### Saving the Convenience DFs

In [49]:
coarse_64d_fv_df.to_csv(str(path_interim_64fv / 'coarse_summary.csv'), index=False)

In [50]:
coarse_i3d_df.to_csv(str(path_interim_i3d / 'coarse_summary.csv'), index=False)

## Creating One Dataset

In [51]:
raw_data_path = DIR_RAW_BREAKFAST / f'{n_frames}_frame_clips'
if not raw_data_path.exists():
    raw_data_path.mkdir(parents=True)

### 64 Dim FVs

In [53]:
path_64dim_clips = path_interim_64fv / 'clips'
all_clips_data = [[path, np.load(str(path))] for path in tqdm(path_64dim_clips.iterdir())]

33527it [05:59, 93.31it/s] 


In [54]:
path_all_clips, all_data = zip(*all_clips_data)

In [55]:
path_all_clips[:5]

(PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips/36_pancake_webcam02_start_898_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips/51_pancake_webcam02_start_877_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips/30_scrambledegg_cam02_start_1247_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips/41_pancake_cam02_start_2294_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/64dim_fv_clips/clips/26_pancake_webcam02_start_6134_seed_117.npy'))

In [56]:
path_raw_64fv_data = raw_data_path / '64dim_fv'
if not path_raw_64fv_data.exists():
    path_raw_64fv_data.mkdir(parents=True)

In [57]:
with open(str(path_raw_64fv_data / f'path_nonevent_data_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(path_all_clips, filehandle)

In [58]:
all_data_array = np.array(all_data)
all_data_array.shape

(33527, 64, 64)

In [59]:
np.save(str(path_raw_64fv_data / f'nonevent_clips_seed_{SEED}.npy'), all_data_array)

### I3D Dim FVs

In [60]:
path_i3d_clips = path_interim_i3d / 'clips'
all_clips_data = [[path, np.load(str(path))] for path in tqdm(path_i3d_clips.iterdir())]

35699it [35:03, 16.97it/s]


In [61]:
path_all_clips, all_data = zip(*all_clips_data)

In [62]:
path_all_clips[:5]

(PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips/36_pancake_webcam02_start_898_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips/51_pancake_webcam02_start_877_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips/30_scrambledegg_cam02_start_1247_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips/41_pancake_cam02_start_2294_seed_117.npy'),
 PosixPath('/media/data_cifs2/apra/work/labwork/data/interim/breakfast/video_clips/nonevent_clips/i3d_clips/clips/26_pancake_webcam02_start_6134_seed_117.npy'))

In [63]:
path_raw_i3d_data = raw_data_path / 'i3d_fv'
if not path_raw_i3d_data.exists():
    path_raw_i3d_data.mkdir(parents=True)

In [64]:
with open(str(path_raw_i3d_data / f'path_nonevent_data_seed_{SEED}.pkl'), 'wb') as filehandle:
    pickle.dump(path_all_clips, filehandle)

In [66]:
all_data_array = np.array(all_data)
all_data_array.shape

44.9 s ± 5.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
np.save(str(path_raw_i3d_data / f'nonevent_clips_seed_{SEED}.npy'), all_data_array)